# Settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [3]:
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


In [4]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


# Dataset loading

In [5]:
dataset_name='pokemon'

In [6]:
images_dir = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
validation_percentage = 20
valid_format = 'png'

In [7]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [8]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [9]:
inputs_shape= image_size=(200, 200, 3)
batch_size = 10
latents_dim = 32
intermediate_dim = 50

In [10]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode=None
)

  INFO     | Found 662 training files


  INFO     | Found 147 validation files


In [11]:
import tensorflow as tf

In [12]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)

In [13]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [14]:
_instance_scale

1.0

In [15]:
import numpy as np
from collections.abc import Iterable

In [16]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [17]:
_outputs_shape

120000

# Model's Layers definition

In [18]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [19]:
model_name = dataset_name+'AE_Dense_reconst_ell_struct'
experiments_dir='experiments'+sep_local+model_name

In [20]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [21]:
inputs_shape=image_size

In [22]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [23]:
from utils.data_and_files.file_utils import create_if_not_exist

In [24]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [25]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonAE_Dense_reconst_ell_struct\\var_save_dir'

In [26]:
#to restore trained model, set filepath=_restore

In [27]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=_restore
    )

  DEBUG    | Restore old models ...


SavedModel file does not exist at: experiments\pokemonAE_Dense_reconst_ell_struct\var_save_dir\pokemonAE_Dense_reconst_ell_struct_inference.hdf5/{saved_model.pbtxt|saved_model.pb}




  ERROR    | Faild to restore old models !


Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 50)      200       
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 50)      2550      
_________________________________________________________________
flatten (Flatten)            (None, 2000000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                64000032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                1650      
_________________________________________________________________
dense_5 (Dense)              (None, 120000)            6120000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [28]:
#from evaluation.quantitive_metrics.peak_signal_to_noise_ratio import prepare_psnr
from statistical.ultimate_loss import prepare_ell_struct

In [29]:
ae.compile(loss={'x_logits': prepare_ell_struct([ae.batch_size]+ae.get_inputs_shape())})

Model: "pokemonAE_Dense_reconst_ell_struct"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                64002910  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       6122718   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 70,125,628
Trainable params: 70,125,558
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [30]:

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [31]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [32]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [33]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonAE_Dense_reconst_ell_struct\\csv_dir\\pokemonAE_Dense_reconst_ell_struct.csv'

In [34]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [35]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [36]:
import numpy as np

# Model Training

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=1000,
    epochs=int(1e6), 
    verbose=1,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1000
)

Train for 1000 steps, validate for 1000 steps
Epoch 1/1000000
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


  45/1000 [>.............................] - ETA: 2:19:47 - loss: -164955.5469 - psnr: 7.4071 - total_variation: 223466.7188 - ssim_multiscale: 1.1095 - sharpdiff: 7.0431 - mean_absolute_error: 0.3909 - mean_squared_error: 0.183 - ETA: 1:12:06 - loss: 758326.3516 - psnr: 6.0704 - total_variation: 731489.1250 - ssim_multiscale: 0.7539 - sharpdiff: 3.5188 - mean_absolute_error: 0.4175 - mean_squared_error: 0.260 - ETA: 49:32 - loss: 1258094.3177 - psnr: 5.6901 - total_variation: 897173.9375 - ssim_multiscale: 0.6605 - sharpdiff: 2.3561 - mean_absolute_error: 0.4221 - mean_squared_error: 0.2809 - ETA: 38:12 - loss: 1449791.0508 - psnr: 5.6159 - total_variation: 936662.0000 - ssim_multiscale: 0.6130 - sharpdiff: 1.9809 - mean_absolute_error: 0.4164 - mean_squared_error: 0.284 - ETA: 31:25 - loss: 1568894.4656 - psnr: 5.6583 - total_variation: 962267.0000 - ssim_multiscale: 0.5986 - sharpdiff: 1.7463 - mean_absolute_error: 0.4085 - mean_squared_error: 0.281 - ETA: 26:53 - loss: 1690289.6380

  90/1000 [=>............................] - ETA: 7:01 - loss: 1853933.8658 - psnr: 5.8614 - total_variation: 982684.9375 - ssim_multiscale: 0.5518 - sharpdiff: 1.2382 - mean_absolute_error: 0.3758 - mean_squared_error: 0.26 - ETA: 6:57 - loss: 1849380.2729 - psnr: 5.8713 - total_variation: 980065.1875 - ssim_multiscale: 0.5516 - sharpdiff: 1.2508 - mean_absolute_error: 0.3752 - mean_squared_error: 0.26 - ETA: 6:53 - loss: 1842809.6188 - psnr: 5.8791 - total_variation: 976240.6875 - ssim_multiscale: 0.5501 - sharpdiff: 1.2693 - mean_absolute_error: 0.3747 - mean_squared_error: 0.26 - ETA: 6:49 - loss: 1845333.2720 - psnr: 5.8709 - total_variation: 978055.5000 - ssim_multiscale: 0.5483 - sharpdiff: 1.2597 - mean_absolute_error: 0.3750 - mean_squared_error: 0.26 - ETA: 6:46 - loss: 1842624.8216 - psnr: 5.8702 - total_variation: 978665.6875 - ssim_multiscale: 0.5492 - sharpdiff: 1.2553 - mean_absolute_error: 0.3749 - mean_squared_error: 0.26 - ETA: 6:42 - loss: 1839507.1584 - psnr: 5.8780

 135/1000 [===>..........................] - ETA: 5:23 - loss: 1811990.8237 - psnr: 5.8657 - total_variation: 986353.5625 - ssim_multiscale: 0.5456 - sharpdiff: 1.2073 - mean_absolute_error: 0.3744 - mean_squared_error: 0.26 - ETA: 5:22 - loss: 1809830.8161 - psnr: 5.8682 - total_variation: 985967.3750 - ssim_multiscale: 0.5450 - sharpdiff: 1.2088 - mean_absolute_error: 0.3743 - mean_squared_error: 0.26 - ETA: 5:21 - loss: 1807994.9000 - psnr: 5.8624 - total_variation: 985991.0625 - ssim_multiscale: 0.5440 - sharpdiff: 1.2076 - mean_absolute_error: 0.3745 - mean_squared_error: 0.26 - ETA: 5:20 - loss: 1807682.1671 - psnr: 5.8612 - total_variation: 986900.2500 - ssim_multiscale: 0.5444 - sharpdiff: 1.2033 - mean_absolute_error: 0.3746 - mean_squared_error: 0.26 - ETA: 5:19 - loss: 1807756.1390 - psnr: 5.8585 - total_variation: 987983.6875 - ssim_multiscale: 0.5436 - sharpdiff: 1.1981 - mean_absolute_error: 0.3747 - mean_squared_error: 0.26 - ETA: 5:17 - loss: 1806978.2104 - psnr: 5.8590

 180/1000 [====>.........................] - ETA: 4:41 - loss: 1722948.8112 - psnr: 5.8725 - total_variation: 992809.8750 - ssim_multiscale: 0.5547 - sharpdiff: 1.1710 - mean_absolute_error: 0.3741 - mean_squared_error: 0.26 - ETA: 4:40 - loss: 1720441.9987 - psnr: 5.8734 - total_variation: 992712.7500 - ssim_multiscale: 0.5547 - sharpdiff: 1.1715 - mean_absolute_error: 0.3740 - mean_squared_error: 0.26 - ETA: 4:39 - loss: 1719200.1138 - psnr: 5.8694 - total_variation: 994156.3125 - ssim_multiscale: 0.5549 - sharpdiff: 1.1646 - mean_absolute_error: 0.3742 - mean_squared_error: 0.26 - ETA: 4:39 - loss: 1717264.9151 - psnr: 5.8672 - total_variation: 993759.3125 - ssim_multiscale: 0.5546 - sharpdiff: 1.1669 - mean_absolute_error: 0.3744 - mean_squared_error: 0.26 - ETA: 4:38 - loss: 1713596.8309 - psnr: 5.8703 - total_variation: 993181.2500 - ssim_multiscale: 0.5538 - sharpdiff: 1.1696 - mean_absolute_error: 0.3742 - mean_squared_error: 0.26 - ETA: 4:37 - loss: 1711491.5564 - psnr: 5.8681

...

 495/1000 [=============>................] - ETA: 2:35 - loss: 809199.5861 - psnr: 5.5545 - total_variation: 1115491.5000 - ssim_multiscale: 0.5816 - sharpdiff: 0.6233 - mean_absolute_error: 0.3979 - mean_squared_error: 0.28 - ETA: 2:34 - loss: 807369.0086 - psnr: 5.5518 - total_variation: 1115813.2500 - ssim_multiscale: 0.5817 - sharpdiff: 0.6221 - mean_absolute_error: 0.3981 - mean_squared_error: 0.28 - ETA: 2:34 - loss: 804695.1723 - psnr: 5.5517 - total_variation: 1116231.2500 - ssim_multiscale: 0.5816 - sharpdiff: 0.6204 - mean_absolute_error: 0.3981 - mean_squared_error: 0.28 - ETA: 2:34 - loss: 802688.3235 - psnr: 5.5498 - total_variation: 1116654.6250 - ssim_multiscale: 0.5815 - sharpdiff: 0.6186 - mean_absolute_error: 0.3983 - mean_squared_error: 0.28 - ETA: 2:33 - loss: 800499.7590 - psnr: 5.5533 - total_variation: 1115905.1250 - ssim_multiscale: 0.5817 - sharpdiff: 0.6221 - mean_absolute_error: 0.3981 - mean_squared_error: 0.28 - ETA: 2:33 - loss: 798700.8094 - psnr: 5.5505 

 540/1000 [===============>..............] - ETA: 2:21 - loss: 710195.4455 - psnr: 5.5213 - total_variation: 1124961.7500 - ssim_multiscale: 0.5848 - sharpdiff: 0.5866 - mean_absolute_error: 0.4006 - mean_squared_error: 0.28 - ETA: 2:21 - loss: 708840.0895 - psnr: 5.5192 - total_variation: 1125308.7500 - ssim_multiscale: 0.5849 - sharpdiff: 0.5851 - mean_absolute_error: 0.4008 - mean_squared_error: 0.28 - ETA: 2:20 - loss: 706193.6370 - psnr: 5.5183 - total_variation: 1125775.8750 - ssim_multiscale: 0.5851 - sharpdiff: 0.5832 - mean_absolute_error: 0.4008 - mean_squared_error: 0.28 - ETA: 2:20 - loss: 703795.2950 - psnr: 5.5172 - total_variation: 1126130.0000 - ssim_multiscale: 0.5853 - sharpdiff: 0.5817 - mean_absolute_error: 0.4009 - mean_squared_error: 0.28 - ETA: 2:20 - loss: 701170.7049 - psnr: 5.5182 - total_variation: 1126200.2500 - ssim_multiscale: 0.5855 - sharpdiff: 0.5813 - mean_absolute_error: 0.4009 - mean_squared_error: 0.28 - ETA: 2:20 - loss: 699458.2543 - psnr: 5.5163 

 585/1000 [================>.............] - ETA: 2:08 - loss: 619043.2782 - psnr: 5.4965 - total_variation: 1133642.2500 - ssim_multiscale: 0.5886 - sharpdiff: 0.5523 - mean_absolute_error: 0.4028 - mean_squared_error: 0.28 - ETA: 2:07 - loss: 618200.1380 - psnr: 5.4940 - total_variation: 1133895.7500 - ssim_multiscale: 0.5886 - sharpdiff: 0.5511 - mean_absolute_error: 0.4030 - mean_squared_error: 0.28 - ETA: 2:07 - loss: 615695.7815 - psnr: 5.4938 - total_variation: 1134267.3750 - ssim_multiscale: 0.5886 - sharpdiff: 0.5495 - mean_absolute_error: 0.4030 - mean_squared_error: 0.28 - ETA: 2:07 - loss: 613584.1880 - psnr: 5.4926 - total_variation: 1134715.2500 - ssim_multiscale: 0.5886 - sharpdiff: 0.5477 - mean_absolute_error: 0.4031 - mean_squared_error: 0.28 - ETA: 2:07 - loss: 611407.5679 - psnr: 5.4906 - total_variation: 1135027.2500 - ssim_multiscale: 0.5886 - sharpdiff: 0.5465 - mean_absolute_error: 0.4032 - mean_squared_error: 0.29 - ETA: 2:06 - loss: 609080.4863 - psnr: 5.4892 

 630/1000 [=================>............] - ETA: 1:55 - loss: 535426.5529 - psnr: 5.4727 - total_variation: 1142680.3750 - ssim_multiscale: 0.5913 - sharpdiff: 0.5166 - mean_absolute_error: 0.4047 - mean_squared_error: 0.29 - ETA: 1:54 - loss: 533835.4443 - psnr: 5.4715 - total_variation: 1143043.3750 - ssim_multiscale: 0.5912 - sharpdiff: 0.5152 - mean_absolute_error: 0.4048 - mean_squared_error: 0.29 - ETA: 1:54 - loss: 531786.1782 - psnr: 5.4712 - total_variation: 1143395.1250 - ssim_multiscale: 0.5911 - sharpdiff: 0.5137 - mean_absolute_error: 0.4048 - mean_squared_error: 0.29 - ETA: 1:54 - loss: 530309.1090 - psnr: 5.4694 - total_variation: 1143705.3750 - ssim_multiscale: 0.5912 - sharpdiff: 0.5124 - mean_absolute_error: 0.4049 - mean_squared_error: 0.29 - ETA: 1:54 - loss: 528228.1784 - psnr: 5.4690 - total_variation: 1143979.7500 - ssim_multiscale: 0.5914 - sharpdiff: 0.5113 - mean_absolute_error: 0.4049 - mean_squared_error: 0.29 - ETA: 1:53 - loss: 526414.9051 - psnr: 5.4706 

 654/1000 [==================>...........] - ETA: 1:42 - loss: 458567.9502 - psnr: 5.4575 - total_variation: 1151363.3750 - ssim_multiscale: 0.5936 - sharpdiff: 0.4836 - mean_absolute_error: 0.4060 - mean_squared_error: 0.29 - ETA: 1:42 - loss: 456846.1388 - psnr: 5.4575 - total_variation: 1151642.3750 - ssim_multiscale: 0.5936 - sharpdiff: 0.4825 - mean_absolute_error: 0.4060 - mean_squared_error: 0.29 - ETA: 1:41 - loss: 455086.3160 - psnr: 5.4581 - total_variation: 1151740.1250 - ssim_multiscale: 0.5935 - sharpdiff: 0.4820 - mean_absolute_error: 0.4060 - mean_squared_error: 0.29 - ETA: 1:41 - loss: 453611.5184 - psnr: 5.4611 - total_variation: 1151337.3750 - ssim_multiscale: 0.5938 - sharpdiff: 0.4838 - mean_absolute_error: 0.4058 - mean_squared_error: 0.29 - ETA: 1:41 - loss: 452101.2388 - psnr: 5.4599 - total_variation: 1151630.2500 - ssim_multiscale: 0.5938 - sharpdiff: 0.4827 - mean_absolute_error: 0.4059 - mean_squared_error: 0.29 - ETA: 1:40 - loss: 450658.9523 - psnr: 5.4582 

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)